In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import time
from business_bayes import bayesian_search

## Bayesian Search Theory

* [The problem](#The-problem)
* [A concrete example](#A-concrete-example)
    * [Problem specification](#Problem-specification)
    * [Updating our probabilities after performing a search](#Updating-our-probabilities-after-performing-a-search)
    * [Choosing the next box to search](#Choosing-the-next-box-to-search)
    * [The algorithm](#The-algorithm)
    * [Does it help?](#Does-it-help?)
* [Benchmark against random search](#Benchmark-against-random-search)

[Bayesian search theory](https://en.wikipedia.org/wiki/Bayesian_search_theory) is very useful, and it's not just for locating nuclear subs. When can you apply it?

### The problem

* The search space has been divided into $n$ *disjoint* sets: the object is located in one (and only one) of these
* We have a prior belief of what the probabilities are of finding the object in each box
* For each box, we know the probability of finding the object after searching the box *given the object is actually in that box*
* For each box, we know how much performing a search of that box costs

Given all that, **how should we choose the sequence of boxes to search to minimize the costs of finding the object?**

The idea is to iteratively:
* use our probability estimate for the object location, together with the search costs, to choose the next box to search
* use [Bayes' theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) to revise our probability estimates of where the object is every time we perform a search of a box


### A concrete example

We've noticed a radical dropoff of sales in our online store which we're feverishly trying to debug; we're pretty sure it's either a new bug in the backend, a new bug in the frontend, or something else.

Should we start panicking and running around like headless chickens, or is there a more systematic way to approach the search for the problem?


#### Problem specification

##### the search space

So our assumption is there's three possible causes for the slump in sales; we assume only one of these is happening.

1. a bug in the backend code
2. a bug in the frontend code
3. something else

##### the prior probability

We don't have good reasons to believe any one of these is more likely than any of the others, so we'll start by assigning equal probabilities to the three options:

* $P(backend) = \frac{1}{3}$
* $P(frontend) = \frac{1}{3}$
* $P(other) = \frac{1}{3}$

In [2]:
p0 = np.ones(3) / 3.

##### the probability of finding the object in a box given the object is actually there

We're more experienced as a backend developer, so we'll rate our chances of finding a backend bug higher than those of finding a frontend bug by visual inspection. We do not know how to search for "something else", so we rate our chances of finding the cause there by thinking about all possible other causes to be low:

* $P(found \mid backend) = 0.4$
* $P(found \mid fronted) = 0.1$
* $P(found \mid other) = 0.05$

In [3]:
p_found_given_box = np.array([0.4, 0.1, 0.05])

##### the cost of searching each box

Let's say we work in chunks of 1 hour: the cost of looking over the frontend code and the backend code in that case is the same, 1, as is the cost of blue-sky thinking about other possible causes for an hour.

* $cost\_searching\_backend = 1.$
* $cost\_searching\_frontend = 1.$
* $cost\_searching\_other = 1.$

In [4]:
costs = np.ones(3)

#### Updating our probabilities after performing a search

How do our initial probabilities $p_0$ change after, say, poring over the backend code for an hour and **not** having found the bug? 

Intuitively, we know we now have a somewhat lower belief in there being a backend bug, and a somewhat higher belief in the bug being caused by the frontend or by something else.

If the assumptions we've made until now hold, we can use Bayes to exactly quantify **how much** these beliefs should change, let's do the math in the concrete case of having searched for an hour in the backend code and not having found anything.

```
P(A|B) = P(B|A)P(A) / P(B)

P(backend_found | backend) = 0.6
P(backend_not_found | backend) = 1 - P(backend_found | backend) = 0.6

P(backend_not_found | backend) = 0.6
P(backend_not_found | frontend) = 1
P(backend_not_found | other) = 1

P(backend_not_found) = P(backend_not_found | backend) * P(backend) + P(frontend) + P(other)
```

So the posterior probabilities, after searching in the backend code for 1 hour and not finding anything:

```
P(backend | backend_not_found) = P(backend_not_found | backend) * P(backend) / P(backend_not_found)
P(frontend | backend_not_found) = P(frontend) / P(backend_not_found)
P(other | backend_not_found) = P(other) / P(backend_not_found)
```

The bayesian update is implemented by the `bayesian_search.observe` function:

In [5]:
# we've searched in the backend code, which is the box with index 0:
search0 = 0

# get updated probabilities after taking this information into account
p1 = bayesian_search.observe(p0, search0, p_found_given_box[search0])
print("p1: ", p1)

p1:  [0.23076923 0.38461538 0.38461538]


This result is a quantified version of our intuition: we're now less confident that there's a backend problem.

#### Choosing the next box to search

Given:

* the probabilities $p$ representing our current belief of where the object is
* the probabilities $p\_found$ characterizing the effectiveness of our search
* the costs $costs$ of each type of search

what box should I choose to search next?

David Blackwell answered that question already: we should search the box with the index $i$ that maximizes the following quantity:

$$
\frac{p_i * {p\_found}_i}{costs_i}
$$

**Boom**, next, haha. 

I haven't taken the time to actually do it but I'm pretty sure the [Bellman equation](https://en.wikipedia.org/wiki/Bellman_equation) for this should be straightforward: we can use our current probability estimate to take an expectation over the outcomes of the next experiment.

This choice is implemented by the `bayesian_search.choose_next` function:

In [6]:
search1 = bayesian_search.choose_next(p1, p_found_given_box, costs)
print("search1: ", search1)

search1:  0


#### The algorithm

Now we have all the ingredients needed for the Bayesian search algorithm, in pseudopython:

```python
found = False

while True:
    next_box_idx = bayesian_search.choose_next(p, p_found_given_box, costs)
    found = perform_search(next_box_idx)
    if found:
        # yay
        break
    else:
        # update our probabilities
        p = bayesian_search.observe(p, next_box_idx, p_found_given_box[next_box_idx])
```

Let's simulate this.

First, we need to create a problem instance: a `location_idx` represents the true location of the object, a `perform_search` function stochastically performs a search:

In [7]:
# say the real cause is a frontend bug
location_idx = 1

def perform_search(next_box_idx):
    """
    Stand-in for performing a search: return True if found, False otherwise
    """
    if next_box_idx == location_idx:
        prob = p_found_given_box[location_idx]
        found = np.random.binomial(1, prob)
        return found == 1
    else:
        return False

Now we can use our simulated `perform_search` function to simulate the bayesian search, looking at how our probabilities evolve as we perform searches and take the information into account:

In [8]:
np.random.seed(0)
p = p0
cost = 0
while 1:
    next_box_idx = bayesian_search.choose_next(p, p_found_given_box, costs)
    print("probabilities: ", p, "next: ", next_box_idx)
    cost += costs[next_box_idx]
    found = perform_search(next_box_idx)
    if found:
        break
    else:
        p = bayesian_search.observe(p, next_box_idx, p_found_given_box[next_box_idx])
print('total cost: ', cost)

probabilities:  [0.33333333 0.33333333 0.33333333] next:  0
probabilities:  [0.23076923 0.38461538 0.38461538] next:  0
probabilities:  [0.15254237 0.42372881 0.42372881] next:  0
probabilities:  [0.09747292 0.45126354 0.45126354] next:  1
probabilities:  [0.1020794  0.42533081 0.47258979] next:  1
probabilities:  [0.10661402 0.39980257 0.49358342] next:  0
probabilities:  [0.0668179  0.41761188 0.51557022] next:  1
probabilities:  [0.0697299  0.39223071 0.53803938] next:  1
probabilities:  [0.07257658 0.36741894 0.56000448] next:  1
probabilities:  [0.07534489 0.34329018 0.58136493] next:  1
probabilities:  [0.07802336 0.31994454 0.6020321 ] next:  1
probabilities:  [0.08060218 0.29746739 0.62193042] next:  0
probabilities:  [0.04997247 0.30737751 0.64265002] next:  2
probabilities:  [0.05163152 0.31758222 0.63078626] next:  1
probabilities:  [0.05332502 0.295199   0.65147598] next:  2
probabilities:  [0.05512051 0.30513852 0.63974097] next:  2
probabilities:  [0.05694191 0.31522152 0

#### Does it help?

The probability of finding the bug via a random check is:

$$
0.1 * \frac{1}{3} = 0.0\overline{3}
$$

[hence the expected cost of random search](https://en.wikipedia.org/wiki/Geometric_series#Geometric_power_series) works out to be, rather neatly:

$$
cost(steps=1) * prob(steps=1) + cost(steps=2) * prob(steps=2) + \ldots 
$$


$$
p + 2 (1 - p) p + 3 (1 - p) ^ 2 p \ldots k (1 -p) ^ {k -1} p \ldots
$$

$$
p \cdot \sum_{k=1}^{\infty} k \cdot (1 - p) ^ {k -1} = p \cdot \frac{1}{(1 - (1 -p)) ^ 2}
$$

$$
\frac{1}{p}
$$

$$
1 \cdot \frac{1}{0.0\overline{3}} = 30
$$

Let's get a monte carlo estimate of the expected cost in this case with Bayesian search (and with random search to double check my math):

In [9]:
bayes_costs = [bayesian_search.simulate_bayes(p0, p_found_given_box, costs, location_idx) for _ in range(10000)]
print('expected bayesian cost: ', np.mean(bayes_costs), np.std(bayes_costs))
rnd_costs = [bayesian_search.simulate_random(p_found_given_box, costs, location_idx) for _ in range(10000)]
print('expected random cost: ', np.mean(rnd_costs), np.std(rnd_costs))

expected bayesian cost:  24.4034 27.688110597149816
expected random cost:  29.9432 29.481549717747196


In this case we expect to take **~5.5 hours less** if we search using bayesian search than if we just use random search. That's a very respectable 18% less time.

### Benchmark against random search

Intuitivelty, bayesian search helps least for uniformly-distributed priors, costs and finding probabilities: in that case, searching randomly is worse, but not MUCH worse.

Let's test out a few cases against the uniform case.

In [10]:
cases = {
    'uniform': {
        'p0': np.ones(3) / 3,
        'p_found_given_box': np.ones(3) / 5,
        'costs': np.ones(3)
    },
    'skewed_find_probs': {
        'p0': np.ones(3) / 3,
        'p_found_given_box': np.array([100, 10, 1]) / 111,
        'costs': np.ones(3)
    },
    'skewed_costs': {
        'p0': np.ones(3) / 3,
        'p_found_given_box': np.ones(3) / 5,
        'costs': np.array([100, 10, 1]) / 111
    },
    'skewed_all': {
        'p0': np.ones(3) / 3,
        'p_found_given_box': np.array([100, 10, 1]) / 111,
        'costs': np.array([100, 10, 1]) / 111
    },
}

In [11]:
res = {}

for case_name, case in cases.items():
    p0 = case['p0']
    p_found_given_box = case['p_found_given_box']
    costs = case['costs']
    for location_idx in range(3):
        b_samples = [bayesian_search.simulate_bayes(p0, p_found_given_box, costs, location_idx) for _ in range(10000)]
        r_samples = [bayesian_search.simulate_random(p_found_given_box, costs, location_idx) for _ in range(10000)]
        
        res[case_name] = {
            'bayes': b_samples,
            'random': r_samples
        }
    

In [12]:
for case_name, data in res.items():
    print(case_name)
    for t in ['bayes', 'random']:
        print('expected {0} cost: '.format(t), np.mean(data[t]))
    print()

uniform
expected bayes cost:  13.2274
expected random cost:  14.9548

skewed_find_probs
expected bayes cost:  149.4608
expected random cost:  336.1934

skewed_costs
expected bayes cost:  0.12318108108108108
expected random cost:  4.875884684684685

skewed_all
expected bayes cost:  3.001727027027026
expected random cost:  111.84202612612613



Looks like bayesian search can really exploit the probability and costs asymmetries.

So if your search problem is highly asymmetrical, odds are bayesian search can make a huge difference over unprincipled approaches.